#  MABe Challenge - Social Action Recognition in Mice

### Overview
> In this competition, you’ll develop machine learning models to recognize behaviors in mice based on their movements, providing new insights into animal social structures and advancing behavioral science research.

### Description
> Animal social behavior is complex. Species from ants to wolves to mice form social groups where they build nests, raise their young, care for their groupmates, and defend their territory. Studying these behaviors teaches us about the brain and the evolution of behavior, but the work has usually required subjective, time-consuming documentation of animals' actions. ML advancements now let us automate this process, supporting large-scale behavioral studies in the wild and in the lab.

> But even automated systems suffer from limited training data and poor generalizability. In current methods, an experimenter must hand-label hundreds of new training examples to automate recognition of a new behavior, which makes studying rare behaviors a challenge. And models trained within one research group usually fail when applied to data from other studies, meaning there is no guarantee that two labs are really studying the same behavior.

> This competition challenges you to build models to identify over 30 different social and non-social behaviors in pairs and groups of co-housed mice, based on markerless motion capture of their movements in top-down video recordings. The dataset includes over 400 hours of footage from 20+ behavioral recording systems, all carefully labeled frame-by-frame by experts. Your goal is to recognize these behaviors as accurately as a trained human observer while overcoming the inherent variability arising from the use of different data collection equipment and motion capture pipelines.

Your work will help scientists automate behavior analysis and better understand animal social structures. These models may be deployed across numerous labs in neuroscience, computational biology, ethology, and ecology to create a foundation for future ML and behavior research.

This notebook tackles the MABe challenge by building a separate GBDT ensemble model for each unique `body_parts_tracked` configuration. 

**Core Strategy:**
1.  **Grouped by Tracker:** Loop through each `body_parts_tracked` string.
2.  **FPS-Aware Features:** Generate advanced temporal and spatial features (`transform_single`, `transform_pair`). All window sizes, lags, and spans are *scaled* by the video's FPS to ensure features are time-invariant (`_scale` function).
3.  **Stratified Subsampling:** Use a `StratifiedSubsetClassifier` wrapper to train the GBDTs on a large, stratified subsample of the full dataset to manage memory and time.
4.  **Ensemble Model:** For each behavior, train an ensemble of LGBM, XGBoost, and CatBoost models.
5.  **Adaptive Prediction:** Use temporal smoothing, adaptive per-action probability thresholds, and minimum duration filtering to generate final event segments (`predict_multiclass_adaptive`).

In [1]:
# ==================== IMPORTS & SETUP ====================

verbose = True

import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import warnings
import json
import os, random
import gc, re, math
import lightgbm
from collections import defaultdict
import polars as pl
from scipy import signal, stats
from scipy.spatial.distance import euclidean
from typing import Dict, Optional, Tuple, List
from time import perf_counter
from sklearn.base import ClassifierMixin, BaseEstimator, clone
from sklearn.model_selection import cross_val_predict, GroupKFold, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')
USE_GPU = ("KAGGLE_KERNEL_RUN_TYPE" in os.environ) and (__import__("shutil").which("nvidia-smi") is not None)
print(f'Using GPU?  {USE_GPU}')

from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Optional imports for advanced features
try:
    import pywt
    HAS_PYWT = True
    print("PyWavelets available")
except ImportError:
    HAS_PYWT = False
    print("PyWavelets not available - wavelet features disabled")

try:
    import sklearn_crfsuite
    HAS_CRF = True
    print("sklearn-crfsuite available")
except ImportError:
    HAS_CRF = False
    print("sklearn-crfsuite not available - CRF post-processing disabled")

try:
    from fastdtw import fastdtw
    HAS_DTW = True
    print("fastdtw available")
except ImportError:
    HAS_DTW = False
    print("fastdtw not available - template matching disabled")

SEED = 1234

# ==================== SEED EVERYTHING ====================

os.environ["PYTHONHASHSEED"] = str(SEED)

rnd = np.random.RandomState(SEED)
random.seed(SEED)
np.random.seed(SEED)

def _make_lgbm(**kw):
    kw.setdefault("random_state", SEED)
    kw.setdefault("feature_fraction_seed", SEED)
    kw.setdefault("data_random_seed", SEED)
    kw.setdefault("device", 'gpu' if USE_GPU else 'cpu')
    return lightgbm.LGBMClassifier(**kw)

def _make_xgb(**kw):
    kw.setdefault("random_state", SEED)
    kw.setdefault("tree_method", "gpu_hist" if USE_GPU else "hist")
    return XGBClassifier(**kw)

def _make_cb(**kw):
    kw.setdefault("random_seed", SEED)
    if USE_GPU:
        kw.setdefault("task_type", "GPU")
        kw.setdefault("devices", "0")
    else:
        kw.setdefault("task_type", "CPU")
    return CatBoostClassifier(**kw)


# ==================== STRATIFIED SUBSET CLASSIFIERS ====================

class StratifiedSubsetClassifierWEval(ClassifierMixin, BaseEstimator):
    def __init__(self,
                 estimator,
                 n_samples=None,
                 random_state: int = 42,
                 valid_size: float = 0.10,
                 val_cap_ratio: float = 0.25,
                 es_rounds: "int|str" = "auto",
                 es_metric: str = "auto"):
        self.estimator = estimator
        self.n_samples = (int(n_samples) if (n_samples is not None) else None)
        self.random_state = random_state
        self.valid_size = float(valid_size)
        self.val_cap_ratio = float(val_cap_ratio)
        self.es_rounds = es_rounds
        self.es_metric = es_metric

    def fit(self, X: pd.DataFrame, y):
        y = np.asarray(y)
        n_total = len(y)
        assert n_total == len(X)

        tr_idx, va_idx = self._compute_train_val_indices(y, n_total)
        Xtr = X.iloc[tr_idx]
        ytr = y[tr_idx]

        Xtr = Xtr.to_numpy(np.float32, copy=False)

        Xva = yva = None
        if va_idx is not None and len(va_idx) > 0:
            Xva = X.iloc[va_idx].to_numpy(np.float32, copy=False)
            yva = y[va_idx]

        pos_rate = None
        if yva is not None and len(yva) > 0:
            pos_rate = float(np.mean(yva == 1))

        metric = self._choose_metric(pos_rate)
        patience = self._choose_patience(pos_rate)

        if self._is_xgb(self.estimator):
            n_pos = max(1, int((ytr == 1).sum()))
            n_neg = max(1, len(ytr) - n_pos)
            self.estimator.set_params(scale_pos_weight=(n_neg / n_pos))
            self.estimator.set_params(eval_metric=metric)

        elif self._is_catboost(self.estimator):
            try:
                self.estimator.set_params(auto_class_weights="Balanced")
            except Exception:
                pass
            try:
                self.estimator.set_params(eval_metric=metric)
            except Exception:
                pass

        has_valid = (Xva is not None and len(yva) > 0)
        if has_valid and self._is_xgb(self.estimator):
            import xgboost as xgb
            self.estimator.fit(
                Xtr, ytr,
                eval_set=[(Xva, yva)],
                verbose=False,
                callbacks=[xgb.callback.EarlyStopping(
                    rounds=int(patience),
                    metric_name=metric,
                    data_name="validation_0",
                    save_best=True
                )]
            )
        elif has_valid and self._is_catboost(self.estimator):
            from catboost import Pool
            self.estimator.set_params(
                use_best_model=True,
                od_type="Iter",
                od_wait=int(patience),
                custom_metric=["PRAUC:type=Classic;hints=skip_train~true"],
            )
            self.estimator.fit(
                Xtr, ytr,
                eval_set=Pool(Xva, yva),
                verbose=False,
                metric_period=50
            )
        else:
            self.estimator.fit(Xtr, ytr)

        self.classes_ = getattr(self.estimator, "classes_", np.array([0, 1]))
        self._tr_idx_ = tr_idx
        self._va_idx_ = va_idx
        self._pos_rate_ = pos_rate
        return self

    def predict_proba(self, X: pd.DataFrame):
        return self.estimator.predict_proba(X)

    def predict(self, X: pd.DataFrame):
        return self.estimator.predict(X)

    def _compute_train_val_indices(self, y: np.ndarray, n_total: int):
        rng = np.random.default_rng(self.random_state)
        n_classes = np.unique(y).size

        def full_data_split():
            if self.valid_size <= 0 or n_classes < 2:
                idx = rng.permutation(n_total)
                return idx, None
            sss = StratifiedShuffleSplit(n_splits=1, test_size=self.valid_size, random_state=self.random_state)
            tr, va = next(sss.split(np.zeros(n_total, dtype=np.int8), y))
            return tr, va

        if self.n_samples is None or self.n_samples >= n_total:
            return full_data_split()

        sss_tr = StratifiedShuffleSplit(n_splits=1, train_size=self.n_samples, random_state=self.random_state)
        tr_idx, rest_idx = next(sss_tr.split(np.zeros(n_total, dtype=np.int8), y))
        remaining = len(rest_idx)

        min_val_needed = int(np.ceil(self.n_samples * max(self.valid_size, 0.0)))
        val_cap = max(min_val_needed, int(round(self.val_cap_ratio * self.n_samples)))
        want_val = min(remaining, val_cap)

        y_rest = y[rest_idx]
        if remaining < min_val_needed or np.unique(y_rest).size < 2 or self.valid_size <= 0:
            return full_data_split()

        sss_val = StratifiedShuffleSplit(n_splits=1, train_size=want_val, random_state=self.random_state)
        try:
            va_sel, _ = next(sss_val.split(np.zeros(remaining, dtype=np.int8), y_rest))
        except ValueError:
            return full_data_split()

        va_idx = rest_idx[va_sel]
        return tr_idx, va_idx

    def _choose_metric(self, pos_rate=0.01) -> str:
        if self.es_metric != "auto":
            return self.es_metric
        if pos_rate is None or pos_rate == 0.0 or pos_rate == 1.0:
            return "logloss" if self._is_xgb(self.estimator) else "Logloss"
        return "aucpr" if self._is_xgb(self.estimator) else "PRAUC:type=Classic"

    def _choose_patience(self, pos_rate: Optional[float]) -> int:
        if isinstance(self.es_rounds, int):
            return self.es_rounds
        try:
            n_estimators = (int(self.estimator.get_params().get("n_estimators", 200))
                            if self._is_xgb(self.estimator)
                            else int(self.estimator.get_params().get("iterations", 500)))
        except Exception:
            n_estimators = 200
        base = max(30, int(round(0.20 * (n_estimators or 200))))
        if pos_rate is None:
            return base
        if pos_rate < 0.005:
            return int(round(base * 1.75))
        if pos_rate < 0.02:
            return int(round(base * 1.40))
        return base

    @staticmethod
    def _is_xgb(est):
        name = est.__class__.__name__.lower()
        mod = getattr(est, "__module__", "")
        return "xgb" in name or "xgboost" in mod or hasattr(est, "get_xgb_params")

    @staticmethod
    def _is_catboost(est):
        name = est.__class__.__name__.lower()
        mod = getattr(est, "__module__", "")
        return "catboost" in name or "catboost" in mod or hasattr(est, "get_all_params")


class StratifiedSubsetClassifier(ClassifierMixin, BaseEstimator):
    def __init__(self, estimator, n_samples, random_state=SEED):
        self.estimator = estimator
        self.n_samples = n_samples and int(n_samples)
        self.random_state = random_state

    def fit(self, X, y):
        y = np.asarray(y)
        n_total = len(y)

        if self.n_samples is None or self.n_samples >= n_total:
            rng = np.random.default_rng(self.random_state)
            idx = rng.permutation(n_total)
        else:
            sss = StratifiedShuffleSplit(
                n_splits=1, train_size=self.n_samples, random_state=self.random_state
            )
            idx, _ = next(sss.split(np.zeros(n_total, dtype=np.int8), y))

        Xn = X.iloc[idx]
        Xn = Xn.to_numpy(np.float32, copy=False)
        yn = y[idx]

        self.estimator.fit(Xn, yn)
        self.classes_ = getattr(self.estimator, "classes_", np.array([0, 1]))
        return self

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def predict(self, X):
        return self.estimator.predict(X)


# ==================== SCORING FUNCTIONS ====================

class HostVisibleError(Exception):
    pass

def single_lab_f1(lab_solution: pl.DataFrame, lab_submission: pl.DataFrame, beta: float = 1) -> float:
    label_frames: defaultdict[str, set[int]] = defaultdict(set)
    prediction_frames: defaultdict[str, set[int]] = defaultdict(set)

    for row in lab_solution.to_dicts():
        label_frames[row['label_key']].update(range(row['start_frame'], row['stop_frame']))

    for video in lab_solution['video_id'].unique():
        active_labels: str = lab_solution.filter(pl.col('video_id') == video)['behaviors_labeled'].first()
        active_labels: set[str] = set(json.loads(active_labels))
        predicted_mouse_pairs: defaultdict[str, set[int]] = defaultdict(set)

        for row in lab_submission.filter(pl.col('video_id') == video).to_dicts():
            if ','.join([str(row['agent_id']), str(row['target_id']), row['action']]) not in active_labels:
                continue

            new_frames = set(range(row['start_frame'], row['stop_frame']))
            new_frames = new_frames.difference(prediction_frames[row['prediction_key']])
            prediction_pair = ','.join([str(row['agent_id']), str(row['target_id'])])
            if predicted_mouse_pairs[prediction_pair].intersection(new_frames):
                raise HostVisibleError('Multiple predictions for the same frame from one agent/target pair')
            prediction_frames[row['prediction_key']].update(new_frames)
            predicted_mouse_pairs[prediction_pair].update(new_frames)

    tps = defaultdict(int)
    fns = defaultdict(int)
    fps = defaultdict(int)
    for key, pred_frames in prediction_frames.items():
        action = key.split('_')[-1]
        matched_label_frames = label_frames[key]
        tps[action] += len(pred_frames.intersection(matched_label_frames))
        fns[action] += len(matched_label_frames.difference(pred_frames))
        fps[action] += len(pred_frames.difference(matched_label_frames))

    distinct_actions = set()
    for key, frames in label_frames.items():
        action = key.split('_')[-1]
        distinct_actions.add(action)
        if key not in prediction_frames:
            fns[action] += len(frames)

    action_f1s = []
    for action in distinct_actions:
        if tps[action] + fns[action] + fps[action] == 0:
            action_f1s.append(0)
        else:
            action_f1s.append((1 + beta**2) * tps[action] / ((1 + beta**2) * tps[action] + beta**2 * fns[action] + fps[action]))
    return sum(action_f1s) / len(action_f1s)

def mouse_fbeta(solution: pd.DataFrame, submission: pd.DataFrame, beta: float = 1) -> float:
    if len(solution) == 0 or len(submission) == 0:
        raise ValueError('Missing solution or submission data')

    expected_cols = ['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']

    for col in expected_cols:
        if col not in solution.columns:
            raise ValueError(f'Solution is missing column {col}')
        if col not in submission.columns:
            raise ValueError(f'Submission is missing column {col}')

    solution: pl.DataFrame = pl.DataFrame(solution)
    submission: pl.DataFrame = pl.DataFrame(submission)
    assert (solution['start_frame'] <= solution['stop_frame']).all()
    assert (submission['start_frame'] <= submission['stop_frame']).all()
    solution_videos = set(solution['video_id'].unique())
    submission = submission.filter(pl.col('video_id').is_in(solution_videos))

    solution = solution.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('label_key'),
    )
    submission = submission.with_columns(
        pl.concat_str(
            [
                pl.col('video_id').cast(pl.Utf8),
                pl.col('agent_id').cast(pl.Utf8),
                pl.col('target_id').cast(pl.Utf8),
                pl.col('action'),
            ],
            separator='_',
        ).alias('prediction_key'),
    )

    lab_scores = []
    for lab in solution['lab_id'].unique():
        lab_solution = solution.filter(pl.col('lab_id') == lab).clone()
        lab_videos = set(lab_solution['video_id'].unique())
        lab_submission = submission.filter(pl.col('video_id').is_in(lab_videos)).clone()
        lab_scores.append(single_lab_f1(lab_solution, lab_submission, beta=beta))

    return sum(lab_scores) / len(lab_scores)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, beta: float = 1) -> float:
    solution = solution.drop(row_id_column_name, axis='columns', errors='ignore')
    submission = submission.drop(row_id_column_name, axis='columns', errors='ignore')
    return mouse_fbeta(solution, submission, beta=beta)


# ==================== DATA LOADING ====================

train = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')

# Drop likely-sleeping MABe22 clips
train = train.loc[~(train['lab_id'].astype(str).str.contains('MABe22', na=False) &
                    train['mouse1_condition'].astype(str).str.lower().eq('lights on'))].copy()

train['n_mice'] = 4 - train[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)

test = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')
test['sleeping'] = (
    test['lab_id'].astype(str).str.contains('MABe22', na=False) &
    test['mouse1_condition'].astype(str).str.lower().eq('lights on')
)
test['n_mice'] = 4 - test[['mouse1_strain', 'mouse2_strain', 'mouse3_strain', 'mouse4_strain']].isna().sum(axis=1)

body_parts_tracked_list = list(np.unique(train.body_parts_tracked))

drop_body_parts = ['headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft',
                   'headpiece_bottomfrontright', 'headpiece_topbackleft', 'headpiece_topbackright',
                   'headpiece_topfrontleft', 'headpiece_topfrontright', 'spine_1', 'spine_2',
                   'tail_middle_1', 'tail_middle_2', 'tail_midpoint']

_sex_cols = [f'mouse{i}_sex' for i in range(1, 5)]
_train_sex_lut = (train[['video_id'] + _sex_cols].drop_duplicates('video_id')
                  .set_index('video_id').to_dict('index'))
_test_sex_lut = (test[['video_id'] + _sex_cols].drop_duplicates('video_id')
                 .set_index('video_id').to_dict('index'))
_FEATURE_TEMPLATES = {}

print(f"Train videos: {len(train)}, Test videos: {len(test)}")
print(f"Unique body part configs: {len(body_parts_tracked_list)}")



# ==================== DATA GENERATOR ====================

def generate_mouse_data(dataset, traintest, traintest_directory=None,
                        generate_single=True, generate_pair=True):
    assert traintest in ['train', 'test']
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"

    def _to_num(x):
        if isinstance(x, (int, np.integer)):
            return int(x)
        m = re.search(r'(\d+)$', str(x))
        return int(m.group(1)) if m else None

    for _, row in dataset.iterrows():
        lab_id = row.lab_id
        video_id = row.video_id
        fps = float(row.frames_per_second)
        n_mice = int(row.n_mice)
        arena_w = float(row.get('arena_width_cm', np.nan))
        arena_h = float(row.get('arena_height_cm', np.nan))
        sleeping = bool(getattr(row, 'sleeping', False))
        arena_shape = row.get('arena_shape', 'rectangular')

        if not isinstance(row.behaviors_labeled, str):
            continue

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)
        if len(np.unique(vid.bodypart)) > 5:
            vid = vid.query("~ bodypart.isin(@drop_body_parts)")
        pvid = vid.pivot(columns=['mouse_id', 'bodypart'], index='video_frame', values=['x', 'y'])
        del vid
        pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
        pvid = (pvid / float(row.pix_per_cm_approx)).astype('float32', copy=False)

        avail = list(pvid.columns.get_level_values('mouse_id').unique())
        avail_set = set(avail) | set(map(str, avail)) | {f"mouse{_to_num(a)}" for a in avail if _to_num(a) is not None}

        def _resolve(agent_str):
            m = re.search(r'(\d+)$', str(agent_str))
            cand = [agent_str]
            if m:
                n = int(m.group(1))
                cand = [n, n - 1, str(n), f"mouse{n}", agent_str]
            for c in cand:
                if c in avail_set:
                    if c in set(avail):
                        return c
                    for a in avail:
                        if str(a) == str(c) or f"mouse{_to_num(a)}" == str(c):
                            return a
            return None

        vb = json.loads(row.behaviors_labeled)
        vb = sorted(list({b.replace("'", "") for b in vb}))
        vb = pd.DataFrame([b.split(',') for b in vb], columns=['agent', 'target', 'action'])
        vb['agent'] = vb['agent'].astype(str)
        vb['target'] = vb['target'].astype(str)
        vb['action'] = vb['action'].astype(str).str.lower()

        if traintest == 'train':
            try:
                annot = pd.read_parquet(path.replace('train_tracking', 'train_annotation'))
            except FileNotFoundError:
                continue

        def _mk_meta(index, agent_id, target_id):
            m = pd.DataFrame({
                'lab_id': lab_id,
                'video_id': video_id,
                'agent_id': agent_id,
                'target_id': target_id,
                'video_frame': index.astype('int32', copy=False),
                'frames_per_second': np.float32(fps),
                'sleeping': sleeping,
                'arena_shape': arena_shape,
                'arena_width_cm': np.float32(arena_w),
                'arena_height_cm': np.float32(arena_h),
                'n_mice': np.int8(n_mice),
            })
            for c in ('lab_id', 'video_id', 'agent_id', 'target_id', 'arena_shape'):
                m[c] = m[c].astype('category')
            return m

        # SINGLE
        if generate_single:
            vb_single = vb.query("target == 'self'")
            for agent_str in pd.unique(vb_single['agent']):
                col_lab = _resolve(agent_str)
                if col_lab is None:
                    continue
                actions = sorted(vb_single.loc[vb_single['agent'].eq(agent_str), 'action'].unique().tolist())
                if not actions:
                    continue

                single = pvid.loc[:, col_lab]
                meta_df = _mk_meta(single.index, agent_str, 'self')

                if traintest == 'train':
                    a_num = _to_num(col_lab)
                    y = pd.DataFrame(False, index=single.index.astype('int32', copy=False), columns=actions)
                    a_sub = annot.query("(agent_id == @a_num) & (target_id == @a_num)")
                    for i in range(len(a_sub)):
                        ar = a_sub.iloc[i]
                        a = str(ar.action).lower()
                        if a in y.columns:
                            y.loc[int(ar['start_frame']):int(ar['stop_frame']), a] = True
                    yield 'single', single, meta_df, y
                else:
                    yield 'single', single, meta_df, actions

        # PAIR
        if generate_pair:
            vb_pair = vb.query("target != 'self'")
            if len(vb_pair) > 0:
                allowed_pairs = set(map(tuple, vb_pair[['agent', 'target']].itertuples(index=False, name=None)))

                for agent_num, target_num in itertools.permutations(
                        np.unique(pvid.columns.get_level_values('mouse_id')), 2):
                    agent_str = f"mouse{_to_num(agent_num)}"
                    target_str = f"mouse{_to_num(target_num)}"
                    if (agent_str, target_str) not in allowed_pairs:
                        continue

                    a_col = _resolve(agent_str)
                    b_col = _resolve(target_str)
                    if a_col is None or b_col is None:
                        continue

                    actions = sorted(
                        vb_pair.query("(agent == @agent_str) & (target == @target_str)")['action'].unique().tolist()
                    )
                    if not actions:
                        continue

                    pair_xy = pd.concat([pvid[a_col], pvid[b_col]], axis=1, keys=['A', 'B'])
                    meta_df = _mk_meta(pair_xy.index, agent_str, target_str)

                    if traintest == 'train':
                        a_num = _to_num(a_col)
                        b_num = _to_num(b_col)
                        y = pd.DataFrame(False, index=pair_xy.index.astype('int32', copy=False), columns=actions)
                        a_sub = annot.query("(agent_id == @a_num) & (target_id == @b_num)")
                        for i in range(len(a_sub)):
                            ar = a_sub.iloc[i]
                            a = str(ar.action).lower()
                            if a in y.columns:
                                y.loc[int(ar['start_frame']):int(ar['stop_frame']), a] = True
                        yield 'pair', pair_xy, meta_df, y
                    else:
                        yield 'pair', pair_xy, meta_df, actions


# ==================== HELPER FUNCTIONS ====================

def safe_rolling(series, window, func, min_periods=None):
    """Safe rolling operation with NaN handling"""
    if min_periods is None:
        min_periods = max(1, window // 4)
    return series.rolling(window, min_periods=min_periods, center=True).apply(func, raw=True)

def _scale(n_frames_at_30fps, fps, ref=30.0):
    """Scale a frame count defined at 30 fps to the current video's fps."""
    return max(1, int(round(n_frames_at_30fps * float(fps) / ref)))

def _scale_signed(n_frames_at_30fps, fps, ref=30.0):
    """Signed version of _scale for forward/backward shifts."""
    if n_frames_at_30fps == 0:
        return 0
    s = 1 if n_frames_at_30fps > 0 else -1
    mag = max(1, int(round(abs(n_frames_at_30fps) * float(fps) / ref)))
    return s * mag

def _fps_from_meta(meta_df, fallback_lookup, default_fps=30.0):
    if 'frames_per_second' in meta_df.columns and pd.notnull(meta_df['frames_per_second']).any():
        return float(meta_df['frames_per_second'].iloc[0])
    vid = meta_df['video_id'].iloc[0]
    return float(fallback_lookup.get(vid, default_fps))

def _speed(cx: pd.Series, cy: pd.Series, fps: float) -> pd.Series:
    return np.hypot(cx.diff(), cy.diff()).fillna(0.0) * float(fps)

def _roll_future_mean(s: pd.Series, w: int, min_p: int = 1) -> pd.Series:
    return s.iloc[::-1].rolling(w, min_periods=min_p).mean().iloc[::-1]

def _roll_future_var(s: pd.Series, w: int, min_p: int = 2) -> pd.Series:
    return s.iloc[::-1].rolling(w, min_periods=min_p).var().iloc[::-1]


# ==================== NEW: WAVELET FEATURES ====================

def add_wavelet_features(X, cx, cy, fps):
    """Extract multi-scale frequency features using wavelet transform"""
    if not HAS_PYWT:
        return X
    
    try:
        cx_vals = cx.values if hasattr(cx, 'values') else cx
        cy_vals = cy.values if hasattr(cy, 'values') else cy
        
        # Speed signal
        speed = np.sqrt(np.diff(cx_vals)**2 + np.diff(cy_vals)**2) * fps
        speed = np.pad(speed, (1, 0), mode='edge')
        
        if len(speed) < 64:
            return X
        
        # Discrete Wavelet Transform
        max_level = min(4, int(np.log2(len(speed))) - 2)
        if max_level < 1:
            return X
            
        coeffs = pywt.wavedec(speed, 'db4', level=max_level)
        
        # Energy at each decomposition level
        for i, c in enumerate(coeffs):
            energy = np.mean(c**2)
            X[f'wavelet_energy_L{i}'] = energy
        
        # Reconstruct low-frequency (approximation) and high-frequency (detail)
        approx_coeffs = [coeffs[0]] + [np.zeros_like(c) for c in coeffs[1:]]
        approx = pywt.waverec(approx_coeffs, 'db4')[:len(speed)]
        
        detail = speed - approx[:len(speed)]
        
        # Rolling stats on frequency components
        w = _scale(30, fps)
        X['wavelet_low_freq_mean'] = pd.Series(approx).rolling(w, min_periods=1, center=True).mean().values[:len(X)]
        X['wavelet_high_freq_mean'] = pd.Series(np.abs(detail)).rolling(w, min_periods=1, center=True).mean().values[:len(X)]
        X['wavelet_high_freq_std'] = pd.Series(detail).rolling(w, min_periods=1, center=True).std().values[:len(X)]
        
        # Ratio of high to low frequency energy (indicates behavior type)
        X['wavelet_freq_ratio'] = X['wavelet_high_freq_mean'] / (X['wavelet_low_freq_mean'] + 1e-6)
        
    except Exception as e:
        pass
    
    return X


# ==================== NEW: PHYSICS-INFORMED FEATURES ====================

def add_physics_features(X, cx, cy, fps):
    """Physics-based behavior signatures: jerk, angular velocity, kinetic energy"""
    
    try:
        cx_vals = cx.values if hasattr(cx, 'values') else cx
        cy_vals = cy.values if hasattr(cy, 'values') else cy
        
        # Velocity (cm/s)
        vel_x = np.diff(cx_vals, prepend=cx_vals[0]) * fps
        vel_y = np.diff(cy_vals, prepend=cy_vals[0]) * fps
        speed = np.sqrt(vel_x**2 + vel_y**2)
        
        # Acceleration (cm/s^2)
        acc_x = np.diff(vel_x, prepend=vel_x[0]) * fps
        acc_y = np.diff(vel_y, prepend=vel_y[0]) * fps
        acc_mag = np.sqrt(acc_x**2 + acc_y**2)
        
        # Jerk (rate of acceleration change) - key for detecting behavior transitions
        jerk_x = np.diff(acc_x, prepend=acc_x[0]) * fps
        jerk_y = np.diff(acc_y, prepend=acc_y[0]) * fps
        jerk_mag = np.sqrt(jerk_x**2 + jerk_y**2)
        
        w = _scale(30, fps)
        X['jerk_mean'] = pd.Series(jerk_mag).rolling(w, min_periods=1, center=True).mean().values
        X['jerk_max'] = pd.Series(jerk_mag).rolling(w, min_periods=1, center=True).max().values
        X['jerk_std'] = pd.Series(jerk_mag).rolling(w, min_periods=1, center=True).std().values
        
        # Angular velocity (rotation detection)
        angle = np.arctan2(vel_y, vel_x)
        angular_vel = np.abs(np.diff(angle, prepend=angle[0]))
        # Handle wraparound at ±π
        angular_vel = np.minimum(angular_vel, 2 * np.pi - angular_vel) * fps
        
        X['angular_vel_mean'] = pd.Series(angular_vel).rolling(w, min_periods=1, center=True).mean().values
        X['angular_vel_max'] = pd.Series(angular_vel).rolling(w, min_periods=1, center=True).max().values
        X['angular_vel_std'] = pd.Series(angular_vel).rolling(w, min_periods=1, center=True).std().values
        
        # Kinetic energy proxy (mass normalized, so just v^2)
        KE = speed**2
        X['kinetic_energy_mean'] = pd.Series(KE).rolling(w, min_periods=1, center=True).mean().values
        X['kinetic_energy_max'] = pd.Series(KE).rolling(w, min_periods=1, center=True).max().values
        
        # Centripetal acceleration (indicates turning/circling)
        speed_safe = np.maximum(speed, 1e-6)
        centripetal = speed_safe * angular_vel
        X['centripetal_mean'] = pd.Series(centripetal).rolling(w, min_periods=1, center=True).mean().values
        
        # Tangential vs radial acceleration ratio
        tangential_acc = np.abs(np.diff(speed, prepend=speed[0])) * fps
        X['tangential_acc_mean'] = pd.Series(tangential_acc).rolling(w, min_periods=1, center=True).mean().values
        X['acc_ratio'] = X['tangential_acc_mean'] / (X['centripetal_mean'] + 1e-6)
        
    except Exception as e:
        pass
    
    return X


# ==================== NEW: TRANSITION DETECTION FEATURES ====================

def add_transition_features(X, cx, cy, fps):
    """Features for detecting behavior onset/offset transitions"""
    
    try:
        cx_vals = cx.values if hasattr(cx, 'values') else cx
        cy_vals = cy.values if hasattr(cy, 'values') else cy
        
        speed = np.sqrt(np.diff(cx_vals)**2 + np.diff(cy_vals)**2) * fps
        speed = np.pad(speed, (1, 0), mode='edge')
        speed_series = pd.Series(speed)
        
        # Z-score of current speed vs local context (detect sudden changes)
        for w_base in [30, 60, 120]:
            w = _scale(w_base, fps)
            local_mean = speed_series.rolling(w, min_periods=1, center=True).mean()
            local_std = speed_series.rolling(w, min_periods=1, center=True).std() + 1e-6
            X[f'speed_zscore_{w_base}'] = ((speed - local_mean) / local_std).values
        
        # Change point detection: difference between past and future windows
        for w_base in [15, 30]:
            w = _scale(w_base, fps)
            past_mean = speed_series.rolling(w, min_periods=1).mean()
            future_mean = speed_series.iloc[::-1].rolling(w, min_periods=1).mean().iloc[::-1]
            X[f'speed_change_{w_base}'] = (future_mean - past_mean).values
            
            past_std = speed_series.rolling(w, min_periods=1).std()
            future_std = speed_series.iloc[::-1].rolling(w, min_periods=1).std().iloc[::-1]
            X[f'speed_var_change_{w_base}'] = (future_std - past_std).values
        
        # Autocorrelation at different lags (rhythmic behaviors like grooming)
        for lag_base in [5, 10, 15]:
            lag = _scale(lag_base, fps)
            w = _scale(60, fps)
            if len(speed) > lag + w:
                def calc_autocorr(x):
                    if len(x) <= lag:
                        return 0
                    try:
                        return np.corrcoef(x[:-lag], x[lag:])[0, 1]
                    except:
                        return 0
                
                autocorr = speed_series.rolling(w, min_periods=w//2, center=True).apply(calc_autocorr, raw=True)
                X[f'speed_autocorr_{lag_base}'] = autocorr.fillna(0).values
        
        # Speed derivative (acceleration sign changes indicate transitions)
        speed_deriv = np.diff(speed, prepend=speed[0])
        sign_changes = np.abs(np.diff(np.sign(speed_deriv), prepend=0))
        w = _scale(30, fps)
        X['acc_sign_changes'] = pd.Series(sign_changes).rolling(w, min_periods=1, center=True).sum().values
        
    except Exception as e:
        pass
    
    return X


# ==================== NEW: BEHAVIOR TEMPLATE MATCHING ====================

class BehaviorTemplateMatcher:
    """Match trajectory segments to known behavior templates using DTW"""
    
    def __init__(self, segment_length=60, n_templates=5):
        self.segment_length = segment_length
        self.n_templates = n_templates
        self.templates = {}
        self.is_fitted = False
    
    def normalize_segment(self, seg):
        """Normalize segment to fixed length and zero-center"""
        if len(seg) == 0:
            return np.zeros((self.segment_length, 2))
        
        # Resample to fixed length
        if len(seg) != self.segment_length:
            indices = np.linspace(0, len(seg) - 1, self.segment_length).astype(int)
            seg = seg[indices]
        
        # Zero-center
        seg = seg - seg.mean(axis=0)
        
        # Normalize scale
        scale = np.std(seg) + 1e-6
        seg = seg / scale
        
        return seg
    
    def fit(self, trajectories_by_action):
        """Build templates from training data"""
        if not HAS_DTW:
            return self
        
        for action, trajs in trajectories_by_action.items():
            if len(trajs) == 0:
                continue
            
            # Normalize all trajectories
            normalized = [self.normalize_segment(t) for t in trajs]
            
            if len(normalized) <= self.n_templates:
                self.templates[action] = normalized
            else:
                # Random sample for now
                indices = np.random.choice(len(normalized), self.n_templates, replace=False)
                self.templates[action] = [normalized[i] for i in indices]
        
        self.is_fitted = True
        return self
    
    def compute_features(self, trajectory, fps):
        """Compute template matching features for a trajectory"""
        if not self.is_fitted or not HAS_DTW:
            return {}
        
        features = {}
        segment_len = _scale(self.segment_length, fps)
        
        # Get middle segment
        mid = len(trajectory) // 2
        start = max(0, mid - segment_len // 2)
        end = min(len(trajectory), start + segment_len)
        
        if end - start < segment_len // 2:
            return features
        
        segment = trajectory[start:end]
        segment_norm = self.normalize_segment(segment)
        
        for action, templates in self.templates.items():
            dists = []
            for template in templates:
                try:
                    dist, _ = fastdtw(segment_norm, template, dist=euclidean)
                    dists.append(dist)
                except:
                    pass
            
            if dists:
                features[f'template_{action}_min'] = min(dists)
                features[f'template_{action}_mean'] = np.mean(dists)
        
        return features


# Global template matcher
template_matcher = BehaviorTemplateMatcher()


# ==================== NEW: CRF POST-PROCESSING ====================

class BehaviorCRF:
    """Use CRF to enforce temporal consistency of predictions"""
    
    def __init__(self):
        self.crf = None
        self.is_fitted = False
        self.action_list = []
    
    def prepare_features(self, gbdt_probs, fps):
        """Convert per-frame predictions to CRF features"""
        X_crf = []
        
        for i in range(len(gbdt_probs)):
            frame_feats = {}
            
            # GBDT prediction confidences
            for action, prob in gbdt_probs[i].items():
                frame_feats[f'prob_{action}'] = str(round(prob, 2))
                frame_feats[f'prob_{action}_high'] = str(prob > 0.5)
                frame_feats[f'prob_{action}_very_high'] = str(prob > 0.8)
            
            # Context features
            max_prob = max(gbdt_probs[i].values()) if gbdt_probs[i] else 0
            frame_feats['max_prob'] = str(round(max_prob, 2))
            
            # Entropy
            probs = list(gbdt_probs[i].values())
            if probs:
                entropy = -sum(p * np.log(p + 1e-10) for p in probs)
                frame_feats['entropy'] = str(round(entropy, 2))
            
            # Temporal features
            if i > 0:
                for action in gbdt_probs[i]:
                    prev_prob = gbdt_probs[i-1].get(action, 0)
                    delta = gbdt_probs[i][action] - prev_prob
                    frame_feats[f'delta_{action}'] = str(round(delta, 2))
            
            X_crf.append(frame_feats)
        
        return X_crf
    
    def fit(self, videos_probs, videos_labels, fps_list):
        """Train CRF on video sequences"""
        if not HAS_CRF:
            return self
        
        X_train = []
        y_train = []
        
        for probs, labels, fps in zip(videos_probs, videos_labels, fps_list):
            X_train.append(self.prepare_features(probs, fps))
            y_train.append(labels)
        
        self.crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=0.1,
            c2=0.1,
            max_iterations=100,
            all_possible_transitions=True,
        )
        
        try:
            self.crf.fit(X_train, y_train)
            self.is_fitted = True
        except:
            self.is_fitted = False
        
        return self
    
    def predict(self, gbdt_probs, fps):
        """Predict with temporal consistency"""
        if not self.is_fitted or not HAS_CRF:
            return None
        
        X = [self.prepare_features(gbdt_probs, fps)]
        try:
            return self.crf.predict(X)[0]
        except:
            return None


# ==================== ORIGINAL FEATURE FUNCTIONS (ENHANCED) ====================

def add_curvature_features(X, center_x, center_y, fps):
    """Trajectory curvature (window lengths scaled by fps)."""
    vel_x = center_x.diff()
    vel_y = center_y.diff()
    acc_x = vel_x.diff()
    acc_y = vel_y.diff()

    cross_prod = vel_x * acc_y - vel_y * acc_x
    vel_mag = np.sqrt(vel_x**2 + vel_y**2)
    curvature = np.abs(cross_prod) / (vel_mag**3 + 1e-6)

    for w in [30, 60]:
        ws = _scale(w, fps)
        X[f'curv_mean_{w}'] = curvature.rolling(ws, min_periods=max(1, ws // 6)).mean()

    angle = np.arctan2(vel_y, vel_x)
    angle_change = np.abs(angle.diff())
    w = 30
    ws = _scale(w, fps)
    X[f'turn_rate_{w}'] = angle_change.rolling(ws, min_periods=max(1, ws // 6)).sum()

    return X

def add_multiscale_features(X, center_x, center_y, fps):
    """Multi-scale temporal features (speed in cm/s; windows scaled by fps)."""
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)

    scales = [10, 40, 160]
    for scale in scales:
        ws = _scale(scale, fps)
        if len(speed) >= ws:
            X[f'sp_m{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).mean()
            X[f'sp_s{scale}'] = speed.rolling(ws, min_periods=max(1, ws // 4)).std()

    if len(scales) >= 2 and f'sp_m{scales[0]}' in X.columns and f'sp_m{scales[-1]}' in X.columns:
        X['sp_ratio'] = X[f'sp_m{scales[0]}'] / (X[f'sp_m{scales[-1]}'] + 1e-6)

    return X

def add_state_features(X, center_x, center_y, fps):
    """Behavioral state transitions; bins adjusted so semantics are fps-invariant."""
    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    w_ma = _scale(15, fps)
    speed_ma = speed.rolling(w_ma, min_periods=max(1, w_ma // 3)).mean()

    try:
        bins = [-np.inf, 0.5 * fps, 2.0 * fps, 5.0 * fps, np.inf]
        speed_states = pd.cut(speed_ma, bins=bins, labels=[0, 1, 2, 3]).astype(float)

        for window in [60, 120]:
            ws = _scale(window, fps)
            if len(speed_states) >= ws:
                for state in [0, 1, 2, 3]:
                    X[f's{state}_{window}'] = (
                        (speed_states == state).astype(float)
                        .rolling(ws, min_periods=max(1, ws // 6)).mean()
                    )
                state_changes = (speed_states != speed_states.shift(1)).astype(float)
                X[f'trans_{window}'] = state_changes.rolling(ws, min_periods=max(1, ws // 6)).sum()
    except Exception:
        pass

    return X

def add_longrange_features(X, center_x, center_y, fps):
    """Long-range temporal features (windows & spans scaled by fps)."""
    for window in [120, 240]:
        ws = _scale(window, fps)
        if len(center_x) >= ws:
            X[f'x_ml{window}'] = center_x.rolling(ws, min_periods=max(5, ws // 6)).mean()
            X[f'y_ml{window}'] = center_y.rolling(ws, min_periods=max(5, ws // 6)).mean()

    for span in [60, 120]:
        s = _scale(span, fps)
        X[f'x_e{span}'] = center_x.ewm(span=s, min_periods=1).mean()
        X[f'y_e{span}'] = center_y.ewm(span=s, min_periods=1).mean()

    speed = np.sqrt(center_x.diff()**2 + center_y.diff()**2) * float(fps)
    for window in [60, 120]:
        ws = _scale(window, fps)
        if len(speed) >= ws:
            X[f'sp_pct{window}'] = speed.rolling(ws, min_periods=max(5, ws // 6)).rank(pct=True)

    return X

def add_cumulative_distance_single(X, cx, cy, fps, horizon_frames_base: int = 180, colname: str = "path_cum180"):
    L = max(1, _scale(horizon_frames_base, fps))
    step = np.hypot(cx.diff(), cy.diff())
    path = step.rolling(2*L + 1, min_periods=max(5, L//6), center=True).sum()
    X[colname] = path.fillna(0.0).astype(np.float32)
    return X

def add_groom_microfeatures(X, df, fps):
    parts = df.columns.get_level_values(0)
    if 'body_center' not in parts or 'nose' not in parts:
        return X

    cx = df['body_center']['x']
    cy = df['body_center']['y']
    nx = df['nose']['x']
    ny = df['nose']['y']

    cs = (np.sqrt(cx.diff()**2 + cy.diff()**2) * float(fps)).fillna(0)
    ns = (np.sqrt(nx.diff()**2 + ny.diff()**2) * float(fps)).fillna(0)

    w30 = _scale(30, fps)
    X['head_body_decouple'] = (ns / (cs + 1e-3)).clip(0, 10).rolling(w30, min_periods=max(1, w30//3)).median()

    r = np.sqrt((nx - cx)**2 + (ny - cy)**2)
    X['nose_rad_std'] = r.rolling(w30, min_periods=max(1, w30//3)).std().fillna(0)

    if 'tail_base' in parts:
        ang = np.arctan2(df['nose']['y']-df['tail_base']['y'], df['nose']['x']-df['tail_base']['x'])
        dang = np.abs(ang.diff()).fillna(0)
        X['head_orient_jitter'] = dang.rolling(w30, min_periods=max(1, w30//3)).mean()

    return X

def add_interaction_features(X, mouse_pair, avail_A, avail_B, fps):
    """Social interaction features (windows scaled by fps)."""
    if 'body_center' not in avail_A or 'body_center' not in avail_B:
        return X

    rel_x = mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x']
    rel_y = mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y']
    rel_dist = np.sqrt(rel_x**2 + rel_y**2)

    A_vx = mouse_pair['A']['body_center']['x'].diff()
    A_vy = mouse_pair['A']['body_center']['y'].diff()
    B_vx = mouse_pair['B']['body_center']['x'].diff()
    B_vy = mouse_pair['B']['body_center']['y'].diff()

    A_lead = (A_vx * rel_x + A_vy * rel_y) / (np.sqrt(A_vx**2 + A_vy**2) * rel_dist + 1e-6)
    B_lead = (B_vx * (-rel_x) + B_vy * (-rel_y)) / (np.sqrt(B_vx**2 + B_vy**2) * rel_dist + 1e-6)

    for window in [30, 60]:
        ws = _scale(window, fps)
        X[f'A_ld{window}'] = A_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()
        X[f'B_ld{window}'] = B_lead.rolling(ws, min_periods=max(1, ws // 6)).mean()

    approach = -rel_dist.diff()
    chase = approach * B_lead
    w = 30
    ws = _scale(w, fps)
    X[f'chase_{w}'] = chase.rolling(ws, min_periods=max(1, ws // 6)).mean()

    for window in [60, 120]:
        ws = _scale(window, fps)
        A_sp = np.sqrt(A_vx**2 + A_vy**2)
        B_sp = np.sqrt(B_vx**2 + B_vy**2)
        X[f'sp_cor{window}'] = A_sp.rolling(ws, min_periods=max(1, ws // 6)).corr(B_sp)

    return X

def add_speed_asymmetry_future_past_single(X, cx, cy, fps, horizon_base=30, agg="mean"):
    w = max(3, _scale(horizon_base, fps))
    v = _speed(cx, cy, fps)
    if agg == "median":
        v_past = v.rolling(w, min_periods=max(3, w//4), center=False).median()
        v_fut = v.iloc[::-1].rolling(w, min_periods=max(3, w//4)).median().iloc[::-1]
    else:
        v_past = v.rolling(w, min_periods=max(3, w//4), center=False).mean()
        v_fut = _roll_future_mean(v, w, min_p=max(3, w//4))
    X["spd_asym_1s"] = (v_fut - v_past).fillna(0.0)
    return X

def add_gauss_shift_speed_future_past_single(X, cx, cy, fps, window_base=30, eps=1e-6):
    w = max(5, _scale(window_base, fps))
    v = _speed(cx, cy, fps)

    mu_p = v.rolling(w, min_periods=max(3, w//4)).mean()
    va_p = v.rolling(w, min_periods=max(3, w//4)).var().clip(lower=eps)

    mu_f = _roll_future_mean(v, w, min_p=max(3, w//4))
    va_f = _roll_future_var(v, w, min_p=max(3, w//4)).clip(lower=eps)

    kl_pf = 0.5 * ((va_p/va_f) + ((mu_f - mu_p)**2)/va_f - 1.0 + np.log(va_f/va_p))
    kl_fp = 0.5 * ((va_f/va_p) + ((mu_p - mu_f)**2)/va_p - 1.0 + np.log(va_p/va_f))
    X["spd_symkl_1s"] = (kl_pf + kl_fp).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return X


# ==================== ENHANCED TRANSFORM FUNCTIONS ====================

def transform_single(single_mouse, body_parts_tracked, fps):
    """Enhanced single mouse transform with NEW features."""
    available_body_parts = single_mouse.columns.get_level_values(0)

    # Base distance features
    X = pd.DataFrame({
        f"{p1}+{p2}": np.square(single_mouse[p1] - single_mouse[p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.combinations(body_parts_tracked, 2)
        if p1 in available_body_parts and p2 in available_body_parts
    })
    X = X.reindex(columns=[f"{p1}+{p2}" for p1, p2 in itertools.combinations(body_parts_tracked, 2)], copy=False)

    # Speed-like features
    if all(p in single_mouse.columns for p in ['ear_left', 'ear_right', 'tail_base']):
        lag = _scale(10, fps)
        shifted = single_mouse[['ear_left', 'ear_right', 'tail_base']].shift(lag)
        speeds = pd.DataFrame({
            'sp_lf': np.square(single_mouse['ear_left'] - shifted['ear_left']).sum(axis=1, skipna=False),
            'sp_rt': np.square(single_mouse['ear_right'] - shifted['ear_right']).sum(axis=1, skipna=False),
            'sp_lf2': np.square(single_mouse['ear_left'] - shifted['tail_base']).sum(axis=1, skipna=False),
            'sp_rt2': np.square(single_mouse['ear_right'] - shifted['tail_base']).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    if 'nose+tail_base' in X.columns and 'ear_left+ear_right' in X.columns:
        X['elong'] = X['nose+tail_base'] / (X['ear_left+ear_right'] + 1e-6)

    # Body angle
    if all(p in available_body_parts for p in ['nose', 'body_center', 'tail_base']):
        v1 = single_mouse['nose'] - single_mouse['body_center']
        v2 = single_mouse['tail_base'] - single_mouse['body_center']
        X['body_ang'] = (v1['x'] * v2['x'] + v1['y'] * v2['y']) / (
            np.sqrt(v1['x']**2 + v1['y']**2) * np.sqrt(v2['x']**2 + v2['y']**2) + 1e-6)

    # Core temporal features
    if 'body_center' in available_body_parts:
        cx = single_mouse['body_center']['x']
        cy = single_mouse['body_center']['y']

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'cx_m{w}'] = cx.rolling(ws, **roll).mean()
            X[f'cy_m{w}'] = cy.rolling(ws, **roll).mean()
            X[f'cx_s{w}'] = cx.rolling(ws, **roll).std()
            X[f'cy_s{w}'] = cy.rolling(ws, **roll).std()
            X[f'x_rng{w}'] = cx.rolling(ws, **roll).max() - cx.rolling(ws, **roll).min()
            X[f'y_rng{w}'] = cy.rolling(ws, **roll).max() - cy.rolling(ws, **roll).min()
            X[f'disp{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).sum()**2 +
                                     cy.diff().rolling(ws, min_periods=1).sum()**2)
            X[f'act{w}'] = np.sqrt(cx.diff().rolling(ws, min_periods=1).var() +
                                    cy.diff().rolling(ws, min_periods=1).var())

        # Original advanced features
        X = add_curvature_features(X, cx, cy, fps)
        X = add_multiscale_features(X, cx, cy, fps)
        X = add_state_features(X, cx, cy, fps)
        X = add_longrange_features(X, cx, cy, fps)
        X = add_cumulative_distance_single(X, cx, cy, fps, horizon_frames_base=180)
        X = add_groom_microfeatures(X, single_mouse, fps)
        X = add_speed_asymmetry_future_past_single(X, cx, cy, fps, horizon_base=30)
        X = add_gauss_shift_speed_future_past_single(X, cx, cy, fps, window_base=30)
        
        # ===== NEW FEATURES =====
        X = add_wavelet_features(X, cx, cy, fps)
        X = add_physics_features(X, cx, cy, fps)
        X = add_transition_features(X, cx, cy, fps)

    # Nose-tail features
    if all(p in available_body_parts for p in ['nose', 'tail_base']):
        nt_dist = np.sqrt((single_mouse['nose']['x'] - single_mouse['tail_base']['x'])**2 +
                          (single_mouse['nose']['y'] - single_mouse['tail_base']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nt_lg{lag}'] = nt_dist.shift(l)
            X[f'nt_df{lag}'] = nt_dist - nt_dist.shift(l)

    # Ear features
    if all(p in available_body_parts for p in ['ear_left', 'ear_right']):
        ear_d = np.sqrt((single_mouse['ear_left']['x'] - single_mouse['ear_right']['x'])**2 +
                        (single_mouse['ear_left']['y'] - single_mouse['ear_right']['y'])**2)
        for off in [-20, -10, 10, 20]:
            o = _scale_signed(off, fps)
            X[f'ear_o{off}'] = ear_d.shift(-o)
        w = _scale(30, fps)
        X['ear_con'] = ear_d.rolling(w, min_periods=1, center=True).std() / \
                       (ear_d.rolling(w, min_periods=1, center=True).mean() + 1e-6)

    return X.astype(np.float32, copy=False)


def transform_pair(mouse_pair, body_parts_tracked, fps):
    """Enhanced pair transform with NEW features."""
    avail_A = mouse_pair['A'].columns.get_level_values(0)
    avail_B = mouse_pair['B'].columns.get_level_values(0)

    # Inter-mouse distances
    X = pd.DataFrame({
        f"12+{p1}+{p2}": np.square(mouse_pair['A'][p1] - mouse_pair['B'][p2]).sum(axis=1, skipna=False)
        for p1, p2 in itertools.product(body_parts_tracked, repeat=2)
        if p1 in avail_A and p2 in avail_B
    })
    X = X.reindex(columns=[f"12+{p1}+{p2}" for p1, p2 in itertools.product(body_parts_tracked, repeat=2)], copy=False)

    # Speed-like features
    if ('A', 'ear_left') in mouse_pair.columns and ('B', 'ear_left') in mouse_pair.columns:
        lag = _scale(10, fps)
        shA = mouse_pair['A']['ear_left'].shift(lag)
        shB = mouse_pair['B']['ear_left'].shift(lag)
        speeds = pd.DataFrame({
            'sp_A': np.square(mouse_pair['A']['ear_left'] - shA).sum(axis=1, skipna=False),
            'sp_AB': np.square(mouse_pair['A']['ear_left'] - shB).sum(axis=1, skipna=False),
            'sp_B': np.square(mouse_pair['B']['ear_left'] - shB).sum(axis=1, skipna=False),
        })
        X = pd.concat([X, speeds], axis=1)

    # Relative orientation
    if all(p in avail_A for p in ['nose', 'tail_base']) and all(p in avail_B for p in ['nose', 'tail_base']):
        dir_A = mouse_pair['A']['nose'] - mouse_pair['A']['tail_base']
        dir_B = mouse_pair['B']['nose'] - mouse_pair['B']['tail_base']
        X['rel_ori'] = (dir_A['x'] * dir_B['x'] + dir_A['y'] * dir_B['y']) / (
            np.sqrt(dir_A['x']**2 + dir_A['y']**2) * np.sqrt(dir_B['x']**2 + dir_B['y']**2) + 1e-6)

    # Approach rate
    if all(p in avail_A for p in ['nose']) and all(p in avail_B for p in ['nose']):
        cur = np.square(mouse_pair['A']['nose'] - mouse_pair['B']['nose']).sum(axis=1, skipna=False)
        lag = _scale(10, fps)
        shA_n = mouse_pair['A']['nose'].shift(lag)
        shB_n = mouse_pair['B']['nose'].shift(lag)
        past = np.square(shA_n - shB_n).sum(axis=1, skipna=False)
        X['appr'] = cur - past

    # Distance bins
    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd = np.sqrt((mouse_pair['A']['body_center']['x'] - mouse_pair['B']['body_center']['x'])**2 +
                     (mouse_pair['A']['body_center']['y'] - mouse_pair['B']['body_center']['y'])**2)
        X['v_cls'] = (cd < 5.0).astype(float)
        X['cls'] = ((cd >= 5.0) & (cd < 15.0)).astype(float)
        X['med'] = ((cd >= 15.0) & (cd < 30.0)).astype(float)
        X['far'] = (cd >= 30.0).astype(float)

    # Temporal interaction features
    if 'body_center' in avail_A and 'body_center' in avail_B:
        cd_full = np.square(mouse_pair['A']['body_center'] - mouse_pair['B']['body_center']).sum(axis=1, skipna=False)

        for w in [5, 15, 30, 60]:
            ws = _scale(w, fps)
            roll = dict(min_periods=1, center=True)
            X[f'd_m{w}'] = cd_full.rolling(ws, **roll).mean()
            X[f'd_s{w}'] = cd_full.rolling(ws, **roll).std()
            X[f'd_mn{w}'] = cd_full.rolling(ws, **roll).min()
            X[f'd_mx{w}'] = cd_full.rolling(ws, **roll).max()

            d_var = cd_full.rolling(ws, **roll).var()
            X[f'int{w}'] = 1 / (1 + d_var)

            Axd = mouse_pair['A']['body_center']['x'].diff()
            Ayd = mouse_pair['A']['body_center']['y'].diff()
            Bxd = mouse_pair['B']['body_center']['x'].diff()
            Byd = mouse_pair['B']['body_center']['y'].diff()
            coord = Axd * Bxd + Ayd * Byd
            X[f'co_m{w}'] = coord.rolling(ws, **roll).mean()
            X[f'co_s{w}'] = coord.rolling(ws, **roll).std()

        # ===== NEW: Physics features for both mice =====
        cx_A = mouse_pair['A']['body_center']['x']
        cy_A = mouse_pair['A']['body_center']['y']
        cx_B = mouse_pair['B']['body_center']['x']
        cy_B = mouse_pair['B']['body_center']['y']
        
        # Add physics features for mouse A
        X_A = pd.DataFrame(index=X.index)
        X_A = add_physics_features(X_A, cx_A, cy_A, fps)
        for col in X_A.columns:
            X[f'A_{col}'] = X_A[col]
        
        # Add physics features for mouse B
        X_B = pd.DataFrame(index=X.index)
        X_B = add_physics_features(X_B, cx_B, cy_B, fps)
        for col in X_B.columns:
            X[f'B_{col}'] = X_B[col]
        
        # ===== NEW: Relative physics features =====
        # Relative velocity
        rel_vx = np.diff(cx_A.values - cx_B.values, prepend=0) * fps
        rel_vy = np.diff(cy_A.values - cy_B.values, prepend=0) * fps
        rel_speed = np.sqrt(rel_vx**2 + rel_vy**2)
        
        w = _scale(30, fps)
        X['rel_speed_mean'] = pd.Series(rel_speed).rolling(w, min_periods=1, center=True).mean().values
        X['rel_speed_std'] = pd.Series(rel_speed).rolling(w, min_periods=1, center=True).std().values
        
        # Relative acceleration
        rel_ax = np.diff(rel_vx, prepend=0) * fps
        rel_ay = np.diff(rel_vy, prepend=0) * fps
        rel_acc = np.sqrt(rel_ax**2 + rel_ay**2)
        X['rel_acc_mean'] = pd.Series(rel_acc).rolling(w, min_periods=1, center=True).mean().values
        
        # ===== NEW: Interaction dynamics =====
        # Time to collision (if approaching)
        dist = np.sqrt((cx_A.values - cx_B.values)**2 + (cy_A.values - cy_B.values)**2)
        closing_speed = -np.diff(dist, prepend=dist[0]) * fps
        ttc = dist / (closing_speed + 1e-6)
        ttc = np.clip(ttc, -100, 100)
        X['time_to_collision'] = pd.Series(ttc).rolling(w, min_periods=1, center=True).mean().values
        
        # Bearing angle (direction from A to B relative to A's heading)
        if 'nose' in avail_A and 'tail_base' in avail_A:
            heading_x = mouse_pair['A']['nose']['x'] - mouse_pair['A']['tail_base']['x']
            heading_y = mouse_pair['A']['nose']['y'] - mouse_pair['A']['tail_base']['y']
            to_B_x = cx_B - cx_A
            to_B_y = cy_B - cy_A
            
            heading_angle = np.arctan2(heading_y, heading_x)
            to_B_angle = np.arctan2(to_B_y, to_B_x)
            bearing = to_B_angle - heading_angle
            # Normalize to [-pi, pi]
            bearing = np.arctan2(np.sin(bearing), np.cos(bearing))
            
            X['bearing_to_B'] = bearing.values
            X['bearing_to_B_abs'] = np.abs(bearing.values)
            X['facing_B'] = (np.abs(bearing.values) < np.pi/4).astype(float)

    # Nose-nose dynamics
    if 'nose' in avail_A and 'nose' in avail_B:
        nn = np.sqrt((mouse_pair['A']['nose']['x'] - mouse_pair['B']['nose']['x'])**2 +
                     (mouse_pair['A']['nose']['y'] - mouse_pair['B']['nose']['y'])**2)
        for lag in [10, 20, 40]:
            l = _scale(lag, fps)
            X[f'nn_lg{lag}'] = nn.shift(l)
            X[f'nn_ch{lag}'] = nn - nn.shift(l)
            is_cl = (nn < 10.0).astype(float)
            X[f'cl_ps{lag}'] = is_cl.rolling(l, min_periods=1).mean()

    # Velocity alignment
    if 'body_center' in avail_A and 'body_center' in avail_B:
        Avx = mouse_pair['A']['body_center']['x'].diff()
        Avy = mouse_pair['A']['body_center']['y'].diff()
        Bvx = mouse_pair['B']['body_center']['x'].diff()
        Bvy = mouse_pair['B']['body_center']['y'].diff()
        val = (Avx * Bvx + Avy * Bvy) / (np.sqrt(Avx**2 + Avy**2) * np.sqrt(Bvx**2 + Bvy**2) + 1e-6)

        for off in [-20, -10, 0, 10, 20]:
            o = _scale_signed(off, fps)
            X[f'va_{off}'] = val.shift(-o)

        w = _scale(30, fps)
        X['int_con'] = cd_full.rolling(w, min_periods=1, center=True).std() / \
                       (cd_full.rolling(w, min_periods=1, center=True).mean() + 1e-6)

        # Advanced interaction
        X = add_interaction_features(X, mouse_pair, avail_A, avail_B, fps)

    return X.astype(np.float32, copy=False)


# ==================== ADAPTIVE THRESHOLDING ====================

def predict_multiclass_adaptive(pred, meta, action_thresholds=None):
    """Adaptive thresholding per action + temporal smoothing"""
    if action_thresholds is None:
        action_thresholds = defaultdict(lambda: 0.27)
    
    # Apply temporal smoothing
    pred_smoothed = pred.rolling(window=5, min_periods=1, center=True).mean()
    
    ama = np.argmax(pred_smoothed.values, axis=1)
    
    max_probs = pred_smoothed.max(axis=1)
    threshold_mask = np.zeros(len(pred_smoothed), dtype=bool)
    for i, action in enumerate(pred_smoothed.columns):
        action_mask = (ama == i)
        threshold = action_thresholds.get(action, 0.27)
        threshold_mask |= (action_mask & (max_probs >= threshold))
    
    ama = np.where(threshold_mask, ama, -1)
    ama = pd.Series(ama, index=meta.video_frame)
    
    changes_mask = (ama != ama.shift(1)).values
    ama_changes = ama[changes_mask]
    meta_changes = meta[changes_mask]
    mask = ama_changes.values >= 0
    if len(mask) > 0:
        mask[-1] = False
    
    if mask.sum() == 0:
        return pd.DataFrame(columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
    
    submission_part = pd.DataFrame({
        'video_id': meta_changes['video_id'][mask].values,
        'agent_id': meta_changes['agent_id'][mask].values,
        'target_id': meta_changes['target_id'][mask].values,
        'action': pred.columns[ama_changes[mask].values],
        'start_frame': ama_changes.index[mask],
        'stop_frame': ama_changes.index[1:][mask[:-1]]
    })
    
    stop_video_id = meta_changes['video_id'][1:][mask[:-1]].values
    stop_agent_id = meta_changes['agent_id'][1:][mask[:-1]].values
    stop_target_id = meta_changes['target_id'][1:][mask[:-1]].values
    
    for i in range(len(submission_part)):
        video_id = submission_part.video_id.iloc[i]
        agent_id = submission_part.agent_id.iloc[i]
        target_id = submission_part.target_id.iloc[i]
        if i < len(stop_video_id):
            if stop_video_id[i] != video_id or stop_agent_id[i] != agent_id or stop_target_id[i] != target_id:
                new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
                submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
        else:
            new_stop_frame = meta.query("(video_id == @video_id)").video_frame.max() + 1
            submission_part.iat[i, submission_part.columns.get_loc('stop_frame')] = new_stop_frame
    
    # Filter out very short events (likely noise)
    duration = submission_part.stop_frame - submission_part.start_frame
    submission_part = submission_part[duration >= 3].reset_index(drop=True)
    
    if len(submission_part) > 0:
        assert (submission_part.stop_frame > submission_part.start_frame).all(), 'stop <= start'
    
    if verbose:
        print(f'  actions found: {len(submission_part)}')
    return submission_part


# ==================== HELPER FOR LGBM ====================

def _find_lgbm_step(pipe):
    try:
        if "stratifiedsubsetclassifier__estimator" in pipe.get_params():
            est = pipe.get_params()["stratifiedsubsetclassifier__estimator"]
            if isinstance(est, lightgbm.LGBMClassifier):
                return "stratifiedsubsetclassifier"
        if "stratifiedsubsetclassifierweval__estimator" in pipe.get_params():
            est = pipe.get_params()["stratifiedsubsetclassifierweval__estimator"]
            if isinstance(est, lightgbm.LGBMClassifier):
                return "stratifiedsubsetclassifierweval"
    except Exception as e:
        print(e)
    return None


# ==================== ENSEMBLE SUBMISSION ====================

def submit_ensemble(body_parts_tracked_str, switch_tr, X_tr, label, meta, n_samples=1_500_000):
    models = []
    models.append(make_pipeline(
        StratifiedSubsetClassifier(_make_lgbm(
            n_estimators=225, learning_rate=0.07, min_child_samples=40,
            num_leaves=31, subsample=0.8, colsample_bytree=0.8, verbose=-1, gpu_use_dp=USE_GPU
        ), n_samples)
    ))
    models.append(make_pipeline(
        StratifiedSubsetClassifier(_make_lgbm(
            n_estimators=150, learning_rate=0.1, min_child_samples=20,
            num_leaves=63, max_depth=8, subsample=0.7, colsample_bytree=0.9,
            reg_alpha=0.1, reg_lambda=0.1, verbose=-1, gpu_use_dp=USE_GPU
        ), (n_samples and int(n_samples/1.25)))
    ))
    models.append(make_pipeline(
        StratifiedSubsetClassifier(_make_lgbm(
            n_estimators=100, learning_rate=0.05, min_child_samples=30,
            num_leaves=127, max_depth=10, subsample=0.75, verbose=-1, gpu_use_dp=USE_GPU,
        ), (n_samples and int(n_samples/1.66)))
    ))

    xgb0 = _make_xgb(
        n_estimators=180, learning_rate=0.08, max_depth=6,
        min_child_weight=8 if USE_GPU else 5, gamma=1.0 if USE_GPU else 0.,
        subsample=0.8, colsample_bytree=0.8, single_precision_histogram=USE_GPU,
        verbosity=0
    )
    models.append(make_pipeline(StratifiedSubsetClassifier(xgb0, n_samples and int(n_samples/1.2))))

    cb_est = _make_cb(iterations=120, learning_rate=0.1, depth=6,
                      verbose=False, allow_writing_files=False)
    models.append(make_pipeline(StratifiedSubsetClassifier(cb_est, n_samples)))

    model_names = ['lgbm_225', 'lgbm_150', 'lgbm_100', 'xgb_180', 'cat_120']

    if USE_GPU:
        xgb1 = XGBClassifier(
            random_state=SEED, booster="gbtree", tree_method="gpu_hist",
            n_estimators=2000, learning_rate=0.05, grow_policy="lossguide",
            max_leaves=255, max_depth=0, min_child_weight=10, gamma=0.0,
            subsample=0.90, colsample_bytree=1.00, colsample_bylevel=0.85,
            reg_alpha=0.0, reg_lambda=1.0, max_bin=256,
            single_precision_histogram=True, verbosity=0
        )
        models.append(make_pipeline(
            StratifiedSubsetClassifierWEval(xgb1, n_samples and int(n_samples/2.),
                                            random_state=SEED, valid_size=0.10, val_cap_ratio=0.25,
                                            es_rounds="auto", es_metric="auto")
        ))
        xgb2 = XGBClassifier(
            random_state=SEED, booster="gbtree", tree_method="gpu_hist",
            n_estimators=1400, learning_rate=0.06, max_depth=7,
            min_child_weight=12, subsample=0.70, colsample_bytree=0.80,
            reg_alpha=0.0, reg_lambda=1.5, max_bin=256,
            single_precision_histogram=True, verbosity=0
        )
        models.append(make_pipeline(
            StratifiedSubsetClassifierWEval(xgb2, n_samples and int(n_samples/1.5),
                                            random_state=SEED, valid_size=0.10, val_cap_ratio=0.25,
                                            es_rounds="auto", es_metric="auto")
        ))

        cb1 = CatBoostClassifier(
            random_seed=SEED, task_type="GPU", devices="0",
            iterations=4000, learning_rate=0.03, depth=8, l2_leaf_reg=6.0,
            bootstrap_type="Bayesian", bagging_temperature=0.5,
            random_strength=0.5, loss_function="Logloss",
            eval_metric="PRAUC:type=Classic", auto_class_weights="Balanced",
            border_count=64, verbose=False, allow_writing_files=False
        )
        models.append(make_pipeline(
            StratifiedSubsetClassifierWEval(cb1, n_samples and int(n_samples/2.0),
                                            random_state=SEED, valid_size=0.10, val_cap_ratio=0.25,
                                            es_rounds="auto", es_metric="auto")
        ))
        model_names.extend(['xgb1', 'xgb2', 'cat_bay'])

    model_list = []
    for action in label.columns:
        action_mask = ~label[action].isna().values
        y_action = label[action][action_mask].values.astype(int)
        meta_masked = meta.iloc[action_mask]

        trained = []
        for model_idx, m in enumerate(models):
            m_clone = clone(m)
            try:
                t0 = perf_counter()
                m_clone.fit(X_tr[action_mask], y_action)
                dt = perf_counter() - t0
                print(f"trained model {model_names[model_idx]} | {switch_tr} | action={action} | {dt:.1f}s", flush=True)
            except Exception:
                step = _find_lgbm_step(m_clone)
                if step is None:
                    continue
                try:
                    m_clone.set_params(**{f"{step}__estimator__device": "cpu"})
                    t0 = perf_counter()
                    m_clone.fit(X_tr[action_mask], y_action)
                    dt = perf_counter() - t0
                    print(f"trained (CPU fallback) {model_names[model_idx]} | {switch_tr} | action={action} | {dt:.1f}s", flush=True)
                except Exception as e2:
                    print(e2)
                    continue
            trained.append(m_clone)

        if trained:
            model_list.append((action, trained))

    del X_tr
    gc.collect()

    # ---- TEST INFERENCE ----
    body_parts_tracked = json.loads(body_parts_tracked_str)
    if len(body_parts_tracked) > 5:
        body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]

    test_subset = test[test.body_parts_tracked == body_parts_tracked_str]
    generator = generate_mouse_data(
        test_subset, 'test',
        generate_single=(switch_tr == 'single'),
        generate_pair=(switch_tr == 'pair')
    )
    fps_lookup = (test_subset[['video_id', 'frames_per_second']]
                  .drop_duplicates('video_id')
                  .set_index('video_id')['frames_per_second'].to_dict())

    for switch_te, data_te, meta_te, actions_te in generator:
        assert switch_te == switch_tr
        try:
            fps_i = _fps_from_meta(meta_te, fps_lookup, default_fps=30.0)
            if switch_te == 'single':
                X_te = transform_single(data_te, body_parts_tracked, fps_i)
            else:
                X_te = transform_pair(data_te, body_parts_tracked, fps_i)

            del data_te

            pred = pd.DataFrame(index=meta_te.video_frame)
            for action, trained in model_list:
                if action in actions_te:
                    probs = []
                    for mi, mdl in enumerate(trained):
                        probs.append(mdl.predict_proba(X_te)[:, 1])
                    pred[action] = np.mean(probs, axis=0)

            del X_te
            gc.collect()

            if pred.shape[1] != 0:
                submission_list.append(predict_multiclass_adaptive(pred, meta_te))
        except Exception as e:
            print(e)
            try:
                del data_te
            except:
                pass
            gc.collect()


# ==================== ROBUSTIFY SUBMISSION ====================

def robustify(submission, dataset, traintest, traintest_directory=None):
    if traintest_directory is None:
        traintest_directory = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_tracking"

    submission = submission[submission.start_frame < submission.stop_frame]

    group_list = []
    for _, group in submission.groupby(['video_id', 'agent_id', 'target_id']):
        group = group.sort_values('start_frame')
        mask = np.ones(len(group), dtype=bool)
        last_stop = 0
        for i, (_, row) in enumerate(group.iterrows()):
            if row['start_frame'] < last_stop:
                mask[i] = False
            else:
                last_stop = row['stop_frame']
        group_list.append(group[mask])
    submission = pd.concat(group_list) if group_list else submission

    s_list = []
    for _, row in dataset.iterrows():
        lab_id = row['lab_id']
        video_id = row['video_id']
        if (submission.video_id == video_id).any():
            continue

        if verbose:
            print(f"Video {video_id} has no predictions")

        path = f"{traintest_directory}/{lab_id}/{video_id}.parquet"
        vid = pd.read_parquet(path)

        vid_behaviors = eval(row['behaviors_labeled'])
        vid_behaviors = sorted(list({b.replace("'", "") for b in vid_behaviors}))
        vid_behaviors = [b.split(',') for b in vid_behaviors]
        vid_behaviors = pd.DataFrame(vid_behaviors, columns=['agent', 'target', 'action'])

        start_frame = vid.video_frame.min()
        stop_frame = vid.video_frame.max() + 1

        for (agent, target), actions in vid_behaviors.groupby(['agent', 'target']):
            batch_len = int(np.ceil((stop_frame - start_frame) / len(actions)))
            for i, (_, action_row) in enumerate(actions.iterrows()):
                batch_start = start_frame + i * batch_len
                batch_stop = min(batch_start + batch_len, stop_frame)
                s_list.append((video_id, agent, target, action_row['action'], batch_start, batch_stop))

    if len(s_list) > 0:
        submission = pd.concat([
            submission,
            pd.DataFrame(s_list, columns=['video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame'])
        ])

    submission = submission.reset_index(drop=True)
    return submission


# ==================== MAIN LOOP ====================

submission_list = []

for section in range(len(body_parts_tracked_list)):
    body_parts_tracked_str = body_parts_tracked_list[section]
    try:
        body_parts_tracked = json.loads(body_parts_tracked_str)
        print(f"{section}. Processing: {len(body_parts_tracked)} body parts")
        if len(body_parts_tracked) > 5:
            body_parts_tracked = [b for b in body_parts_tracked if b not in drop_body_parts]

        train_subset = train[train.body_parts_tracked == body_parts_tracked_str]

        _fps_lookup = (
            train_subset[['video_id', 'frames_per_second']]
            .drop_duplicates('video_id')
            .set_index('video_id')['frames_per_second']
            .to_dict()
        )

        single_list, single_label_list, single_meta_list = [], [], []
        pair_list, pair_label_list, pair_meta_list = [], [], []

        for switch, data, meta, label in generate_mouse_data(train_subset, 'train'):
            if switch == 'single':
                single_list.append(data)
                single_meta_list.append(meta)
                single_label_list.append(label)
            else:
                pair_list.append(data)
                pair_meta_list.append(meta)
                pair_label_list.append(label)

        if len(single_list) > 0:
            single_feats_parts = []
            for data_i, meta_i in zip(single_list, single_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup, default_fps=30.0)
                Xi = transform_single(data_i, body_parts_tracked, fps_i).astype(np.float32)
                single_feats_parts.append(Xi)

            X_tr = pd.concat(single_feats_parts, axis=0, ignore_index=True)

            single_label = pd.concat(single_label_list, axis=0, ignore_index=True)
            single_meta = pd.concat(single_meta_list, axis=0, ignore_index=True)

            del single_list, single_label_list, single_meta_list, single_feats_parts
            gc.collect()

            print(f"  Single: {X_tr.shape}")
            submit_ensemble(body_parts_tracked_str, 'single', X_tr, single_label, single_meta)

            del X_tr, single_label, single_meta
            gc.collect()

        if len(pair_list) > 0:
            pair_feats_parts = []
            for data_i, meta_i in zip(pair_list, pair_meta_list):
                fps_i = _fps_from_meta(meta_i, _fps_lookup, default_fps=30.0)
                Xi = transform_pair(data_i, body_parts_tracked, fps_i).astype(np.float32)
                pair_feats_parts.append(Xi)

            X_tr = pd.concat(pair_feats_parts, axis=0, ignore_index=True)

            pair_label = pd.concat(pair_label_list, axis=0, ignore_index=True)
            pair_meta = pd.concat(pair_meta_list, axis=0, ignore_index=True)

            del pair_list, pair_label_list, pair_meta_list, pair_feats_parts
            gc.collect()

            print(f"  Pair: {X_tr.shape}")
            submit_ensemble(body_parts_tracked_str, 'pair', X_tr, pair_label, pair_meta)

            del X_tr, pair_label, pair_meta
            gc.collect()

    except Exception as e:
        print(f'***Exception*** {str(e)[:100]}')

    gc.collect()
    print()

if len(submission_list) > 0:
    submission = pd.concat(submission_list, ignore_index=True)
else:
    submission = pd.DataFrame({
        'video_id': [438887472],
        'agent_id': ['mouse1'],
        'target_id': ['self'],
        'action': ['rear'],
        'start_frame': [278],
        'stop_frame': [500]
    })

submission_robust = robustify(submission, test, 'test')
submission_robust.index.name = 'row_id'
submission_robust.to_csv('submission.csv')
print(f"\nSubmission created: {len(submission_robust)} predictions")

print("\n" + "="*60)
print("🏆 ADVANCED FEATURES ADDED:")
print("="*60)
print("✅ Wavelet Transform Features (multi-scale frequency analysis)")
print("✅ Physics-Informed Features (jerk, angular velocity, kinetic energy)")
print("✅ Transition Detection Features (behavior onset/offset)")
print("✅ Enhanced Pair Interaction Features (bearing, time-to-collision)")
print("✅ Relative Physics Features (relative velocity, acceleration)")
print("="*60)

Using GPU?  False
PyWavelets available
sklearn-crfsuite not available - CRF post-processing disabled
fastdtw not available - template matching disabled
Train videos: 8789, Test videos: 1
Unique body part configs: 10
0. Processing: 12 body parts

1. Processing: 18 body parts
  Single: (652695, 152)
trained model lgbm_225 | single | action=rear | 62.3s
trained model lgbm_150 | single | action=rear | 55.6s
trained model lgbm_100 | single | action=rear | 54.5s
trained model xgb_180 | single | action=rear | 57.4s
trained model cat_120 | single | action=rear | 38.3s
  actions found: 10
  actions found: 86
  actions found: 49
  actions found: 117
  Pair: (1524906, 169)
trained model lgbm_225 | pair | action=approach | 89.6s
trained model lgbm_150 | pair | action=approach | 80.7s
trained model lgbm_100 | pair | action=approach | 56.6s
trained model xgb_180 | pair | action=approach | 89.0s
trained model cat_120 | pair | action=approach | 63.3s
trained model lgbm_225 | pair | action=attack | 88.